<a href="https://colab.research.google.com/github/JohnnyLomas/transgenic/blob/main/examples/Transgenic_SingleSequence_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TransGenic: Single Sequence Prediction

To test TransGenic with an example DNA sequence, first load a pretrained model and the input and output tokenizers from HuggingFace.

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer

# Load the model
model_name = "jlomas/HyenaTransgenic-512L9A4-160M"
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Load the output tokenizer
gffTokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Load the input tokenizer
dnaTokenizer = AutoTokenizer.from_pretrained("LongSafari/hyenadna-large-1m-seqlen-hf", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

configuration_transgenic.py:   0%|          | 0.00/3.02k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/jlomas/HyenaTransgenic-512L9A4-160M:
- configuration_transgenic.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_HeynaTransgenic.py:   0%|          | 0.00/28.2k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

tokenization_hyena.py:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/LongSafari/hyenadna-large-1m-seqlen-hf:
- tokenization_hyena.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


special_tokens_map.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

Tokenize an input DNA sequence.

In [2]:
# AT1G58150.TAIR10 gene sequence
seq = 'GCTTATGTTTATCTTTTGATCTGATCTATAAATATATATACAGGTTATCAAAAGGCCTCCACCAAAACCAACTCAACATCTCCGCCTCCATCTCCGCCTCCATCTCCGCCGCGAGTTCCAGACGCTCAAGAATTGGAGTACCTTAAATCCGACTCTTTTCCCGAACACGATGCGTAGAGTTGTCATTCGGACGGAGGTGTGCGTTCCGATAAAATTAGGCTACCGCCGCGGCTTTCAGACCTTCTAGAATTGGAGAAATTGTTTCCCGAACGCGAGGCGCTGAGTTGTCCTTTGGACGGAGATGAGGATTCCAATGAACTTAGGCTACGGCCGCTGGTTCCAGACGCTCAAGAATGGAAGTACCCTAAATCCAAGTTATTTCCCAGACACGCGGCGTGGAGTTGTCATTCGGGCGGAGGTGGAGGTGGAGGCGGTGGCCGTGTATTTACAAATAAAGTAAATGCGGTAGAAGAATTCAACTTAGGAGGACTGAAGGACAGCGAATCCGATTCCGATTCCGAGTAGGGAACTTTTAAAACAACTTTGATTATGGATTTCGATATCCAGAATAATTTTAATTCACTGCTGTTGGACTTGATTAATTTCCTATCACATAACGTTTTGGTTTAACTTTGTACGACCACCA'

# Tokenize the input sequences and remove the [SEP] token
seqs = dnaTokenizer.batch_encode_plus(
    [seq],
    return_tensors="pt")["input_ids"][:, :-1]

Predict the annotation and convert it to GSF format.

In [3]:
model.eval()
if torch.cuda.is_available():
    seqs = seqs.to("cuda")
    model.to("cuda")

# Prediction
outputs = model.generate(
    inputs=seqs,
    num_return_sequences=1,
    max_length=2048,
    num_beams=2,
    do_sample=True,
    decoder_input_ids = None
)

# Convert to GSF
prediction = gffTokenizer.batch_decode(
    outputs.detach().cpu().numpy(),
    skip_special_tokens=True
)

print(prediction)

['<s>2888|CDS1|3257|+|A>CDS1']


Convert the output to GFF for downstream analysis.

In [6]:
! git clone git@github.com:JohnnyLomas/transgenic.git

import os, sys

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), 'transgenic/src')))
from utils.gsf import gffString2GFF3

gff = gffString2GFF3(prediction[0], "Chr5", 1234, "gene_model=AT1G58150.TAIR10")
for i in gff:
	print(i)

Cloning into 'transgenic'...
Host key verification failed.
fatal: Could not read from remote repository.

Please make sure you have the correct access rights
and the repository exists.


ModuleNotFoundError: No module named 'utils'